<a href="https://colab.research.google.com/github/tabaraei/aspect-based-sentiment-analysis/blob/main/ABSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use Born and explanations provided to understand why the opinion is positive or negative, and at the end compare it with pre-trained model.

[Born Classifier](https://bornrule.eguidotti.com) is a text classification algorithm inspired by the notion of superposition of states in quantum
physics. Born provides good classification performance, explainability, and computational efficiency. In this
project, the goal is to exploit the Born explanation in order to use it for Aspect Based Sentiment Analysis. In
particular, the main idea to to proceed as follows:
1. Perform a sentiment analysis classification of documents using Born
2. Extract the explanation features for each pair of documents and predicted labels
3. Analyze the explanatory features in order to group them in candidate aspects
4. Associate each aspect to a specific sentence or portion of the text
5. Predict the sentiment for the sentence or text portion using the trained Born classifier
6. Associate then a (potentially different) sentiment to each sentence or text portion according to the aspect
Finally, evaluate the quality of the results for each aspect.

## Dataset

Any dataset supporting ABSA. See for example [here](https://paperswithcode.com/datasets?task=aspect-based-sentiment-analysis&page=1).

In [ ]:
%%capture
!pip install datasets

In [18]:
from datasets import load_dataset

# dataset = load_dataset("alexcadillon/SemEval2014Task4", "restaurants")
dataset = load_dataset("alexcadillon/SemEval2014Task4", "restaurants")
train_data, test_data = dataset["train"], dataset["test"]

train_data.shape, test_data.shape

((3041, 4), (800, 4))

In [22]:
train_data[0]

{'sentenceId': '3121',
 'text': 'But the staff was so horrible to us.',
 'aspectTerms': [{'term': 'staff',
   'polarity': 'negative',
   'from': '8',
   'to': '13'}],
 'aspectCategories': [{'category': 'service', 'polarity': 'negative'}]}

In [ ]:
sample_text = train_data[2]['text']
sample_text

"The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not."

In [24]:
dataset = [
    {
        'sentenceId': '1634',
        'text': "The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not.",
        'aspectTerms': [
            {'term': 'food', 'polarity': 'positive', 'from': '4', 'to': '8'},
            {'term': 'kitchen', 'polarity': 'positive', 'from': '55', 'to': '62'},
            {'term': 'menu', 'polarity': 'neutral', 'from': '141', 'to': '145'}
        ],
        'aspectCategories': [{'category': 'food', 'polarity': 'positive'}]
    },
    {
        'sentenceId': '1635',
        'text': "The service was slow but the ambiance was pleasant.",
        'aspectTerms': [
            {'term': 'service', 'polarity': 'negative', 'from': '4', 'to': '11'},
            {'term': 'ambiance', 'polarity': 'positive', 'from': '28', 'to': '36'}
        ],
        'aspectCategories': [{'category': 'service', 'polarity': 'negative'}]
    }
]

import pandas as pd

# Flatten aspect terms into a DataFrame
rows = []
for item in dataset:
    text = item['text']
    for aspect in item['aspectTerms']:
        rows.append({
            'sentenceId': item['sentenceId'],
            'text': text,
            'aspect': aspect['term'],
            'polarity': aspect['polarity']
        })

# Create DataFrame
df = pd.DataFrame(rows)
df

,sentenceId,text,aspect,polarity
0,1634,"The food is uniformly exceptional, with a very...",food,positive
1,1634,"The food is uniformly exceptional, with a very...",kitchen,positive
2,1634,"The food is uniformly exceptional, with a very...",menu,neutral
3,1635,The service was slow but the ambiance was plea...,service,negative
4,1635,The service was slow but the ambiance was plea...,ambiance,positive


## Born

In [ ]:
%%capture
!pip install bornrule

In [ ]:
from bornrule import BornClassifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the classifier
clf = BornClassifier()
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

## References

- Emanuele Guidotti and Alfio Ferrara. Text Classification with Born’s Rule. Advances in Neural Information
Processing Systems, 2022.
- Schouten, K., & Frasincar, F. (2015). Survey on aspect-level sentiment analysis. IEEE Transactions on
Knowledge and Data Engineering, 28(3), 813-830. [link](https://ieeexplore.ieee.org/document/7286808)
- Rana, T. A., & Cheah, Y. N. (2016). Aspect extraction in sentiment analysis: comparative analysis and survey.
Artificial Intelligence Review, 46(4), 459-483. [link](https://link.springer.com/article/10.1007/s10462-016-9472-z)